In [ ]:
!pip install simpleai

In [ ]:
import numpy as np
import copy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
import plotly.graph_objects as px
import math
from simpleai.search import SearchProblem, astar
from simpleai.search import SearchProblem, astar, breadth_first, uniform_cost, greedy, limited_depth_first
from skimage.transform import downscale_local_mean

In [ ]:
#------------------------------------------------------------------------------------------------------------------
#   Height map pre-processing
#------------------------------------------------------------------------------------------------------------------

#------------------------------------------------------------------------------------------------------------------
#   Imports
#------------------------------------------------------------------------------------------------------------------
from simpleai.search import SearchProblem, astar, breadth_first,  uniform_cost, greedy, limited_depth_first
import numpy as np
import plotly.graph_objects as go
import time
import math

#------------------------------------------------------------------------------------------------------------------
class PathPlan(SearchProblem):

    def __init__(self, initial_state, goal_state, mars_map, max_height):

        super().__init__(initial_state=initial_state)
        self.goal_state = goal_state
        self.mars_map = mars_map
        self.max_height = max_height

    def actions(self, state):
        i, j = state
        movs = [(0,1), (0,-1), (1,0), (-1,0),
                (1,1), (-1,-1), (1,-1), (-1,1)]
        valid_moves = []
        for di, dj in movs:
            ni, nj = i + di, j + dj
            if 0 <= ni < nr and 0 <= nj < nc:
                if mars_map[i, j] != -1 and mars_map[ni, nj] != -1:
                    if abs(mars_map[i, j] - mars_map[ni, nj]) < max_height:
                        valid_moves.append((ni, nj))
        return valid_moves

    def result(self, state, action):
        return action

    def is_goal(self, state):
        return state == self.goal_state

    def cost(self, state, action, new_state):
        r, c = state
        nr, nc = new_state
        return math.dist((r, c), (nr, nc))

    def heuristic(self, state):
        gr, gr = self.goal_state
        sr, sr = state
        return math.dist((sr, sr), (gr, gr))


#------------------------------------------------------------------------------------------------------------------

mars_map = np.load('/content/map.npy')
nr, nc = mars_map.shape

scale = 10.017636635991

x_origin = 4017
y_origin = 1312
x_desti = 2043
y_desti = 3405

'''
x_origin = 1400
y_origin = 3600
x_desti = 1900
y_desti = 5800

x_origin = 6050
y_origin = 1750
x_desti = 4600
y_desti = 2500

x_origin = 2500
y_origin = 6000
x_desti = 2200
y_desti = 9000

x_origin = 4300
y_origin = 4700
x_desti = 730
y_desti = 9200

x_origin = 2300
y_origin = 1370
x_desti = 4000
y_desti = 11500 '''
max_height = 0.75

def find_positions():
    row_start = nr - round(y_origin / scale)
    col_start = round(x_origin / scale)
    row_goal  = nr - round(y_desti / scale)
    col_goal  = round(x_desti / scale)
    return (row_start, col_start), (row_goal, col_goal)

initial_state, goal_state = find_positions()
print("START =", initial_state, "GOAL =", goal_state)

# Medir el tiempo de ejecución
start_time = time.time()

problem = PathPlan(initial_state, goal_state, mars_map, max_height)
result = astar(problem, graph_search=True)
#result = greedy(problem, graph_search=True)
#result = breadth_first(problem, graph_search=True)
#result = uniform_cost(problem, graph_search=True)

# Calcular el tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time

if result is None:
    print("No se encontró un camino.")
else:
    print("Distance:", scale * result.cost)
    path = [state for state, action in result.path() if state is not None and isinstance(state, tuple) and len(state) == 2]
     # Mostrar el tiempo de ejecución
    print(f"Tiempo de ejecución: {execution_time:.4f} segundos")

print("Camino trazado: ", path)
path_y = [(nr-row) * scale for row, col in path]
path_x = [col * scale for row, col in path]
path_z = [mars_map[row, col]+2 for row, col in path]

x = scale*np.arange(mars_map.shape[1])
y = scale*np.arange(mars_map.shape[0])
X, Y = np.meshgrid(x, y)

fig = go.Figure(data = [go.Surface(x=X, y=Y, z=np.flipud(mars_map), colorscale='hot', cmin = 0,
                                    lighting = dict(ambient = 0.0, diffuse = 0.8, fresnel = 0.02, roughness = 0.4, specular = 0.2),
                                    lightposition=dict(x=0, y=nr/2, z=2*mars_map.max())),

                        go.Scatter3d(x = path_x, y = path_y, z = path_z, name='path', mode='markers',
                                        marker=dict(color=np.linspace(0, 1, len(path_x)), colorscale="viridis", size=4))],

                layout = go.Layout(scene_aspectmode='manual',
                                    scene_aspectratio=dict(x=1, y=nr/nc, z=max(mars_map.max()/x.max(), 0.2)),
                                    scene_zaxis_range = [0,mars_map.max()])
                )
fig.show()

#------------------------------------------------------------------------------------------------------------------
#   End of file
#------------------------------------------------------------------------------------------------------------------

In [ ]:
import heapq
import numpy as np
import math

class BestFirstSearch:
    def __init__(self, problem):
        self.problem = problem

    def search(self):
        # The open list (priority queue) and closed list (visited nodes)
        open_list = []
        closed_list = set()

        # Push the start state into the priority queue
        start_state = self.problem.initial_state
        heapq.heappush(open_list, (self.problem.heuristic(start_state), start_state))

        # To track the path
        came_from = {}

        while open_list:
            # Get the state with the lowest heuristic
            _, current_state = heapq.heappop(open_list)

            if current_state in closed_list:
                continue

            # If it's the goal state, reconstruct the path and return it
            if self.problem.is_goal(current_state):
                path = []
                while current_state in came_from:
                    path.append(current_state)
                    current_state = came_from[current_state]
                path.append(self.problem.initial_state)
                path.reverse()
                return path

            closed_list.add(current_state)

            # Get neighbors of the current state
            for action in self.problem.actions(current_state):
                if action not in closed_list:
                    heapq.heappush(open_list, (self.problem.heuristic(action), action))
                    came_from[action] = current_state

        return None  # If no path found

# Now, use the custom Best-First Search in your main code:

# Modify your problem definition if needed
mars_map = np.load('/content/map.npy')
nr, nc = mars_map.shape

scale = 10.017636635991

x_origin = 2300
y_origin = 1370
x_desti = 4000
y_desti = 11500
max_height = 0.75

def find_positions():
    row_start = nr - round(y_origin / scale)
    col_start = round(x_origin / scale)
    row_goal  = nr - round(y_desti / scale)
    col_goal  = round(x_desti / scale)
    return (row_start, col_start), (row_goal, col_goal)

initial_state, goal_state = find_positions()
print("START =", initial_state, "GOAL =", goal_state)

# Define the PathPlan problem
class PathPlan(SearchProblem):

    def __init__(self, initial_state, goal_state, mars_map, max_height):
        super().__init__(initial_state=initial_state)
        self.goal_state = goal_state
        self.mars_map = mars_map
        self.max_height = max_height

    def actions(self, state):
        i, j = state
        movs = [(0,1), (0,-1), (1,0), (-1,0),
                (1,1), (-1,-1), (1,-1), (-1,1)]
        valid_moves = []
        for di, dj in movs:
            ni, nj = i + di, j + dj
            if 0 <= ni < nr and 0 <= nj < nc:
                if mars_map[i, j] != -1 and mars_map[ni, nj] != -1:
                    if abs(mars_map[i, j] - mars_map[ni, nj]) < max_height:
                        valid_moves.append((ni, nj))
        return valid_moves

    def result(self, state, action):
        return action

    def is_goal(self, state):
        return state == self.goal_state

    def cost(self, state, action, new_state):
        r, c = state
        nr, nc = new_state
        return math.dist((r, c), (nr, nc))

    def heuristic(self, state):
        gr, gc = self.goal_state
        sr, sc = state
        return math.dist((sr, sc), (gr, gc))

# Create the problem instance
problem = PathPlan(initial_state, goal_state, mars_map, max_height)

# Perform the Best-First Search
search = BestFirstSearch(problem)
result = search.search()

# Output results
if result is None:
    print("No path found.")
else:
    print("Path found:", result)

# Medir el tiempo de ejecución
start_time = time.time()

problem = PathPlan(initial_state, goal_state, mars_map, max_height)
result = astar(problem, graph_search=True)


# Calcular el tiempo de ejecución
end_time = time.time()
execution_time = end_time - start_time

if result is None:
    print("No se encontró un camino.")
else:
    print("Distance:", scale * result.cost)
    path = [state for state, action in result.path() if state is not None and isinstance(state, tuple) and len(state) == 2]
     # Mostrar el tiempo de ejecución
    print(f"Tiempo de ejecución: {execution_time:.4f} segundos")

print("Camino trazado: ", path)
path_y = [(nr-row) * scale for row, col in path]
path_x = [col * scale for row, col in path]
path_z = [mars_map[row, col]+2 for row, col in path]

x = scale*np.arange(mars_map.shape[1])
y = scale*np.arange(mars_map.shape[0])
X, Y = np.meshgrid(x, y)

fig = go.Figure(data = [go.Surface(x=X, y=Y, z=np.flipud(mars_map), colorscale='hot', cmin = 0,
                                    lighting = dict(ambient = 0.0, diffuse = 0.8, fresnel = 0.02, roughness = 0.4, specular = 0.2),
                                    lightposition=dict(x=0, y=nr/2, z=2*mars_map.max())),

                        go.Scatter3d(x = path_x, y = path_y, z = path_z, name='path', mode='markers',
                                        marker=dict(color=np.linspace(0, 1, len(path_x)), colorscale="viridis", size=4))],

                layout = go.Layout(scene_aspectmode='manual',
                                    scene_aspectratio=dict(x=1, y=nr/nc, z=max(mars_map.max()/x.max(), 0.2)),
                                    scene_zaxis_range = [0,mars_map.max()])
                )
fig.show()


In [ ]:
import numpy as np
import plotly.graph_objects as go
import time
import math

def backtrack_path(start, goal, mars_map, max_height):
    nr, nc = mars_map.shape
    path = []
    visited = set()
    movements = [(0,1), (0,-1), (1,0), (-1,0), (1,1), (-1,-1), (1,-1), (-1,1)]

    def is_valid_move(i, j, ni, nj):
        return (0 <= ni < nr and 0 <= nj < nc and
                (ni, nj) not in visited and
                mars_map[i, j] != -1 and mars_map[ni, nj] != -1 and
                abs(mars_map[i, j] - mars_map[ni, nj]) < max_height)

    def backtrack(state):
        if state == goal:
            path.append(state)
            return True

        visited.add(state)
        path.append(state)
        i, j = state

        for di, dj in movements:
            ni, nj = i + di, j + dj
            if is_valid_move(i, j, ni, nj):
                if backtrack((ni, nj)):
                    return True

        path.pop()
        return False

    if backtrack(start):
        return path
    else:
        return None

# Load the Mars map
mars_map = np.load('map.npy')
nr, nc = mars_map.shape
scale = 10.017636635991

# Define start and goal positions
x_origin = 4017
y_origin = 1312
x_desti = 2043
y_desti = 3405
max_height = 0.75

def find_positions():
    row_start = nr - round(y_origin / scale)
    col_start = round(x_origin / scale)
    row_goal  = nr - round(y_desti / scale)
    col_goal  = round(x_desti / scale)
    return (row_start, col_start), (row_goal, col_goal)

initial_state, goal_state = find_positions()
print("START =", initial_state, "GOAL =", goal_state)

# Measure execution time
start_time = time.time()
path = backtrack_path(initial_state, goal_state, mars_map, max_height)
execution_time = time.time() - start_time

if path is None:
    print("No path found.")
else:
    print("Path found with", len(path), "steps")
    print(f"Execution time: {execution_time:.4f} seconds")

# Visualization
path_y = [(nr-row) * scale for row, col in path]
path_x = [col * scale for row, col in path]
path_z = [mars_map[row, col] + 2 for row, col in path]

x = scale * np.arange(mars_map.shape[1])
y = scale * np.arange(mars_map.shape[0])
X, Y = np.meshgrid(x, y)

fig = go.Figure(data=[
    go.Surface(x=X, y=Y, z=np.flipud(mars_map), colorscale='hot'),
    go.Scatter3d(x=path_x, y=path_y, z=path_z, mode='markers', marker=dict(color='blue', size=4))
])
fig.show()


START = (1080, 401) GOAL = (871, 204)


RecursionError: maximum recursion depth exceeded in comparison